In [1]:
import numpy as np
import pandas as pd
from hmmlearn import hmm
from sklearn.metrics import confusion_matrix
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans

## This dataset consists of 140K+ NPR radio transcripts (from npr.org) covering 20 years of NPR programs, comprising the transcriptions of 10K+ hours of audio.

## Authors
AUTHOR NAME
Shuyang Li

BIO
UCSD PhD Researcher

## Coverage
TEMPORAL COVERAGE START DATE
01/06/1999

TEMPORAL COVERAGE END DATE
10/10/2019

GEOSPATIAL COVERAGE
Global

## DOI Citation
DOI (DIGITAL OBJECT IDENTIFIER)
10.34740/kaggle/ds/590180

## Provenance
SOURCES
npr.org

COLLECTION METHODOLOGY
Scraped via Python Requests/BeautifulSoup

## License
Data files © Original Authors

## Expected Update Frequency
Never (Updated 3 years ago)

# Model Evaluation

Out dataset is naturally split into episodes. To evaluate our model, we will leave out a certain percentage of episodes to train the model, then test our model on entire episodes to see how successfully we can identify different speakers.

This will be ideal. Training on the same episode would fail to provide independent enough data, giving us too much information with the sentence before and after a test sentence. However, by using separate episodes, we have a considering a separate data-collection session.

In [5]:
df = pd.read_csv('archive/utterances.csv')

In [3]:
display(df.head(10))

,episode,episode_order,speaker,utterance
0,57264,9,"Ms. LOREN MOONEY (Editor-in-Chief, Bicycling M...","It's a 2,200-mile race. To give some sense of ..."
1,57264,10,"Ms. LOREN MOONEY (Editor-in-Chief, Bicycling M...",So for a top competitor like Lance to try to m...
2,57264,11,"NEAL CONAN, host","So in every team, presumably there's one star,..."
3,57264,12,"Ms. LOREN MOONEY (Editor-in-Chief, Bicycling M...",That's right. Each team has nine riders. And w...
4,57264,13,"NEAL CONAN, host","So slipstream, this is like drafting in car ra..."
5,57264,14,"Ms. LOREN MOONEY (Editor-in-Chief, Bicycling M...",That's exactly right.
6,57264,15,"NEAL CONAN, host",And so the guy who's in back has an easier tim...
7,57264,16,"Ms. LOREN MOONEY (Editor-in-Chief, Bicycling M...",That's right. There's a lot of deal making tha...
8,57264,17,"NEAL CONAN, host","We're talking with Loren Mooney, the editor-in..."
9,57264,18,JOHN (Caller),Hello.


In [4]:
# remove all speakers labeled as _NO_SPEAKER
df = df[df['speaker'] != '_NO_SPEAKER']
print(len(df))

# group by episode
episodes = df.groupby('episode')

# show the first episode, 57264
display(episodes.get_group(57264))

3015435


,episode,episode_order,speaker,utterance
0,57264,9,"Ms. LOREN MOONEY (Editor-in-Chief, Bicycling M...","It's a 2,200-mile race. To give some sense of ..."
1,57264,10,"Ms. LOREN MOONEY (Editor-in-Chief, Bicycling M...",So for a top competitor like Lance to try to m...
2,57264,11,"NEAL CONAN, host","So in every team, presumably there's one star,..."
3,57264,12,"Ms. LOREN MOONEY (Editor-in-Chief, Bicycling M...",That's right. Each team has nine riders. And w...
4,57264,13,"NEAL CONAN, host","So slipstream, this is like drafting in car ra..."
...,...,...,...,...
3199423,57264,4,"NEAL CONAN, host",Joining us now from our bureau in New York is ...
3199424,57264,5,"Ms. LOREN MOONEY (Editor-in-Chief, Bicycling M...",Thanks for having me.
3199425,57264,6,"NEAL CONAN, host","And I've got my copy of Bicycling Magazine, an..."
3199426,57264,7,"Ms. LOREN MOONEY (Editor-in-Chief, Bicycling M...","Well, yes, it's true. Actually, the race did b..."
